In [ ]:
import pandas as pd
import numpy as np
import igraph as ig
import scipy

In [ ]:
def jaccard(mat):
    """
    Calculate the Jaccard similarity matrix based on the given sparse matrix.

    Parameters:
        mat (scipy.sparse.csr_matrix): Sparse matrix to calculate the Jaccard similarity.

    Returns:
        scipy.sparse.csr_matrix: Jaccard similarity matrix.
    """
    # Ensure input matrix is in CSR (compressed sparse row) format for efficient computation
    mat = mat.tocsr()

    # Compute the intersection matrix
    intersection = mat.T.dot(mat)

    # Compute the sum of row elements for the union calculation
    row_sum = np.array(mat.sum(axis=1)).flatten()

    # Compute the union matrix using vectorized operations
    union = row_sum + row_sum[intersection.indices] - intersection.data

    # Calculate Jaccard similarity values and set the diagonal to zero
    similarities = intersection.copy().astype(float)
    similarities.data /= union[intersection.indices]
    similarities.setdiag(0)

    return similarities


def clone_calling(jac_mat_low, cells, jac_th=0.7):
    """
    Perform clone calling based on the given Jaccard similarity matrix,
    a list of cells, and a Jaccard similarity threshold.

    Parameters:
        jac_mat_low (ndarray): Jaccard similarity matrix as a numpy ndarray.
        cells (list): List of cell identifiers.
        jac_th (float): Jaccard similarity threshold. Default is 0.7.

    Returns:
        Tuple: A tuple containing the resulting graph and a clone table DataFrame.

    """
    # Convert jac_mat_low to boolean matrix directly
    jac_mat = jac_mat_low > jac_th

    # Construct edge_df numpy array directly
    edge_df = np.column_stack(np.where(jac_mat))
    edge_df = edge_df.astype(str)
    
    # Construct vertex_df numpy array directly
    vertex_df = np.column_stack((np.arange(len(cells)), cells, cells))
    
    g = ig.Graph.DataFrame(edges=edge_df, directed=False, vertices=vertex_df)
    g.delete_vertices(g.vs.select(_degree=0))
    clones = g.components()
    
    edge_den = [g.induced_subgraph(i).density() * len(i) for i in clones]
    clones_bc = [g.induced_subgraph(i).vs['name'] for i in clones]
    clone_id = [[j+1] * len(i) for j, i in enumerate(clones_bc)]
    
    # Create a DataFrame directly from the constructed lists
    clone_table = pd.DataFrame(
        data=np.column_stack((np.concatenate(clone_id),
                              np.concatenate(clones_bc),
                              edge_den)),
        columns=['clone.id', 'cell.bc', 'edge.den']
    )

    return g, clone_table  


In [ ]:
# load the molecular table 
molecular_table = pd.read_csv("/path/to/molecular_table.csv")

In [ ]:
at_piv = pd.pivot_table(
        molecular_table, index="cellBC", columns="intBC", values="UMI", aggfunc="count"
    )
at_piv.fillna(value=0, inplace=True)

In [ ]:
cell_mat = at_piv.astype(pd.SparseDtype("float64",0)).sparse.to_coo().tocsr()
cloneBC = np.array(at_piv.columns)
cells = np.array(at_piv.index)

In [ ]:
cell_mat_bin = cell_mat > 1 # filter cell-intBC combo < 2 UMIs
row_fil = ((cell_mat_bin.sum(axis=1) > 1)) # only filter low UMI cell
temp = cell_mat_bin[row_fil.nonzero()[0],]
col_fil = temp.sum(axis=0) > 5
cell_mat_met = temp[:,col_fil.nonzero()[1]].copy()
cell_mat_met = cell_mat_met*1

cells_met = np.array(cells)[row_fil.nonzero()[0]]
cloneBC_met = np.array(cloneBC)[col_fil.nonzero()[1]]

In [ ]:
jac_mat = jaccard(cell_mat_met.tocsr().astype(np.float64).transpose())
g, clones = clone_calling(scipy.sparse.tril(jac_mat), cells_met, jac_th=0.6)
clones['size'] = clones.groupby('clone.id').transform('size')
clone_info = clones.groupby('clone.id')[("edge.den", "size")].value_counts().reset_index()

In [ ]:
clone_info

,clone.id,edge.den,size,0,ctpc_union,ctpc_core
0,1,1.000000,65,65,1,1
1,2,1.000000,22,22,1,1
2,3,1.000000,9,9,1,1
3,4,0.555556,9,9,4,3
4,5,1.000000,23,23,1,1
...,...,...,...,...,...,...
167,168,1.000000,6,6,1,1
168,169,1.000000,5,5,2,2
169,170,1.000000,3,3,1,1
170,171,1.000000,3,3,1,1


In [ ]:
# save clone callling results
clone_info.to_csv("/path/to/clone_info.csv")